In [1]:
import tensorflow as tf
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from model import FlowNet
from dataset import Scene_Flow_disparity

Instructions for updating:
Use the retry module or similar alternatives.


In [3]:
dataset = Scene_Flow_disparity()

>> already download flyingthings3d__frames_cleanpass.tar of Scene Flow Datasets
>> already extracted flyingthings3d__frames_cleanpass of Scene Flow Datasets
>> already download driving__frames_cleanpass.tar of Scene Flow Datasets
>> already extracted driving__frames_cleanpass of Scene Flow Datasets
>> already download monkaa__frames_cleanpass.tar of Scene Flow Datasets
>> already extracted monkaa__frames_cleanpass of Scene Flow Datasets
>> already download flyingthings3d__frames_finalpass.tar of Scene Flow Datasets
>> already extracted flyingthings3d__frames_finalpass of Scene Flow Datasets
>> already download driving__frames_finalpass.tar of Scene Flow Datasets
>> already extracted driving__frames_finalpass of Scene Flow Datasets
>> already download monkaa__frames_finalpass.tar of Scene Flow Datasets
>> already extracted monkaa__frames_finalpass of Scene Flow Datasets
>> already download flyingthings3d__disparity.tar.bz2 of Scene Flow Datasets
>> already extracted flyingthings3d__disp

In [4]:
print(len(dataset.data_paths))
print()
iteration = 0
for dir_ in dataset.data_paths:
    iteration += 1
    if iteration == 200:
        break
    if dataset.data(dir_) is not None:
        print(dir_)
    else:
        print('no left or png path')

238944

no left or png path
./data/Scene Flow Datasets//flyingthings3d__frames_finalpass/frames_finalpass/TEST/C/0137/left/0010.png
no left or png path
no left or png path
no left or png path
no left or png path
no left or png path
no left or png path
./data/Scene Flow Datasets//flyingthings3d__frames_cleanpass/frames_cleanpass/TRAIN/B/0269/left/0006.png
no left or png path
no left or png path
no left or png path
no left or png path
./data/Scene Flow Datasets//flyingthings3d__frames_cleanpass/frames_cleanpass/TRAIN/B/0437/left/0009.png
./data/Scene Flow Datasets//monkaa__frames_finalpass/frames_finalpass/funnyworld_camera2_augmented1_x2/left/0279.png
no left or png path
no left or png path
./data/Scene Flow Datasets//flyingthings3d__frames_cleanpass/frames_cleanpass/TRAIN/C/0744/left/0014.png
no left or png path
no left or png path
no left or png path
no left or png path
no left or png path
./data/Scene Flow Datasets//monkaa__frames_cleanpass/frames_cleanpass/lonetree_augmented1_x2/lef

./data/Scene Flow Datasets//monkaa__frames_finalpass/frames_finalpass/treeflight_augmented0_x2/left/0301.png
no left or png path
no left or png path
no left or png path
no left or png path
no left or png path
no left or png path
./data/Scene Flow Datasets//flyingthings3d__frames_finalpass/frames_finalpass/TEST/B/0040/left/0009.png
no left or png path
no left or png path
no left or png path
no left or png path
no left or png path
no left or png path
no left or png path
no left or png path
./data/Scene Flow Datasets//flyingthings3d__frames_finalpass/frames_finalpass/TEST/A/0011/left/0008.png
no left or png path
./data/Scene Flow Datasets//monkaa__frames_finalpass/frames_finalpass/lonetree_difftex_x2/left/0101.png


In [ ]:
import time
import datetime

model = FlowNet(
    img_height = 540, 
    img_width = 960, 
    img_depth = 3, 
    learning_rate = 0.1**4)

net = model.inference('simple')

start=time.clock()

input image resized by (height = 512, width = 960)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
left_input (InputLayer)         (None, 512, 960, 3)  0                                            
__________________________________________________________________________________________________
right_image (InputLayer)        (None, 512, 960, 3)  0                                            
__________________________________________________________________________________________________
concate_view (Concatenate)      (None, 512, 960, 6)  0           left_input[0][0]                 
                                                                 right_image[0][0]                
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 256, 480, 64) 18880

In [ ]:
train_left_x = np.array([])
train_left_x_batch = np.array([])
train_right_x = np.array([])
train_right_x_batch = np.array([])
train_y = np.array([])
train_y_batch = np.array([])

iteration = 0
for dir_ in dataset.data_paths:
    if dataset.data(dir_) is not None:
        left_image, right_image, ground_truth = dataset.data(dir_, 
                                                             (model.model_in_height, model.model_in_width), 
                                                             (model.model_out_height, model.model_out_width))
        
        if train_left_x.size == 0:
            train_left_x = left_image[np.newaxis,:]
            train_right_x = right_image[np.newaxis,:]
            train_y = ground_truth[np.newaxis,:]
        else:
            train_left_x_batch = np.append(train_left_x, left_image[np.newaxis,:], axis = 0)
            train_right_x_batch = np.append(train_right_x, right_image[np.newaxis,:], axis = 0)
            train_y_batch = np.append(train_y, ground_truth[np.newaxis,:], axis = 0)
        
        iteration += 1
        
    elif iteration >= 100:
        hist = net.fit([train_left_x_batch, train_right_x_batch], 
                           train_y_batch, 
                           batch_size=8, 
                           epochs=100, 
                           validation_split=0.2)
        net.save_weights('./checkpoints/flownetSimple_for_depth.hdf5')
        train_left_x = np.array([])
        train_left_x_batch = np.array([])
        train_right_x = np.array([])
        train_right_x_batch = np.array([])
        train_y = np.array([])
        train_y_batch = np.array([])
        iteration = 0

Train on 1 samples, validate on 1 samples
Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 107.4998 - val_loss: 113.1085
Epoch 2/100
1/1 [==============================] - 0s 104ms/step - loss: 86.6024 - val_loss: 91.3476
Epoch 3/100
1/1 [==============================] - 0s 101ms/step - loss: 71.6762 - val_loss: 69.4158
Epoch 4/100
1/1 [==============================] - 0s 107ms/step - loss: 61.3277 - val_loss: 47.6288
Epoch 5/100
1/1 [==============================] - 0s 103ms/step - loss: 54.6607 - val_loss: 41.6282
Epoch 6/100
1/1 [==============================] - 0s 101ms/step - loss: 57.1510 - val_loss: 43.9251
Epoch 7/100
1/1 [==============================] - 0s 102ms/step - loss: 59.3670 - val_loss: 42.7167
Epoch 8/100
1/1 [==============================] - 0s 102ms/step - loss: 57.7115 - val_loss: 40.6181
Epoch 9/100
1/1 [==============================] - 0s 102ms/step - loss: 55.3324 - val_loss: 39.2955
Epoch 10/100
1/1 [==============================] 

Epoch 82/100
1/1 [==============================] - 0s 102ms/step - loss: 24.0902 - val_loss: 55.3496
Epoch 83/100
1/1 [==============================] - 0s 105ms/step - loss: 22.3595 - val_loss: 52.0540
Epoch 84/100
1/1 [==============================] - 0s 102ms/step - loss: 21.6038 - val_loss: 51.8981
Epoch 85/100
1/1 [==============================] - 0s 106ms/step - loss: 21.2629 - val_loss: 61.0569
Epoch 86/100
1/1 [==============================] - 0s 103ms/step - loss: 21.9053 - val_loss: 45.8667
Epoch 87/100
1/1 [==============================] - 0s 100ms/step - loss: 25.8108 - val_loss: 56.6030
Epoch 88/100
1/1 [==============================] - 0s 104ms/step - loss: 20.2624 - val_loss: 62.5551
Epoch 89/100
1/1 [==============================] - 0s 102ms/step - loss: 21.5318 - val_loss: 44.2502
Epoch 90/100
1/1 [==============================] - 0s 105ms/step - loss: 27.5871 - val_loss: 48.8357
Epoch 91/100
1/1 [==============================] - 0s 102ms/step - loss: 21.2257 

Epoch 63/100
1/1 [==============================] - 0s 100ms/step - loss: 21.1394 - val_loss: 62.6257
Epoch 64/100
1/1 [==============================] - 0s 100ms/step - loss: 18.5896 - val_loss: 80.1294
Epoch 65/100
1/1 [==============================] - 0s 101ms/step - loss: 20.1733 - val_loss: 73.0915
Epoch 66/100
1/1 [==============================] - 0s 103ms/step - loss: 17.9817 - val_loss: 61.0260
Epoch 67/100
1/1 [==============================] - 0s 104ms/step - loss: 19.2989 - val_loss: 62.1882
Epoch 68/100
1/1 [==============================] - 0s 101ms/step - loss: 18.3233 - val_loss: 77.3922
Epoch 69/100
1/1 [==============================] - 0s 103ms/step - loss: 18.1281 - val_loss: 76.2331
Epoch 70/100
1/1 [==============================] - 0s 104ms/step - loss: 17.6289 - val_loss: 62.7099
Epoch 71/100
1/1 [==============================] - 0s 101ms/step - loss: 17.4262 - val_loss: 62.8179
Epoch 72/100
1/1 [==============================] - 0s 100ms/step - loss: 17.0990 

Epoch 43/100
1/1 [==============================] - 0s 101ms/step - loss: 22.8942 - val_loss: 98.1441
Epoch 44/100
1/1 [==============================] - 0s 102ms/step - loss: 22.5584 - val_loss: 107.8318
Epoch 45/100
1/1 [==============================] - 0s 98ms/step - loss: 23.2328 - val_loss: 99.4192
Epoch 46/100
1/1 [==============================] - 0s 102ms/step - loss: 21.2679 - val_loss: 93.7421
Epoch 47/100
1/1 [==============================] - 0s 99ms/step - loss: 22.5440 - val_loss: 103.3364
Epoch 48/100
1/1 [==============================] - 0s 101ms/step - loss: 21.5520 - val_loss: 102.4717
Epoch 49/100
1/1 [==============================] - 0s 102ms/step - loss: 20.6258 - val_loss: 98.5280
Epoch 50/100
1/1 [==============================] - 0s 101ms/step - loss: 21.6993 - val_loss: 109.1268
Epoch 51/100
1/1 [==============================] - 0s 102ms/step - loss: 20.7791 - val_loss: 106.2888
Epoch 52/100
1/1 [==============================] - 0s 101ms/step - loss: 19.89

Epoch 23/100
1/1 [==============================] - 0s 102ms/step - loss: 15.8388 - val_loss: 75.9174
Epoch 24/100
1/1 [==============================] - 0s 103ms/step - loss: 15.2312 - val_loss: 76.2905
Epoch 25/100
1/1 [==============================] - 0s 103ms/step - loss: 15.2171 - val_loss: 75.7836
Epoch 26/100
1/1 [==============================] - 0s 102ms/step - loss: 15.0249 - val_loss: 74.7410
Epoch 27/100
1/1 [==============================] - 0s 104ms/step - loss: 14.7355 - val_loss: 73.7154
Epoch 28/100
1/1 [==============================] - 0s 108ms/step - loss: 14.7103 - val_loss: 73.4094
Epoch 29/100
1/1 [==============================] - 0s 103ms/step - loss: 14.6299 - val_loss: 73.9424
Epoch 30/100
1/1 [==============================] - 0s 103ms/step - loss: 14.2615 - val_loss: 74.7073
Epoch 31/100
1/1 [==============================] - 0s 103ms/step - loss: 14.0352 - val_loss: 74.9520
Epoch 32/100
1/1 [==============================] - 0s 103ms/step - loss: 13.7983 

...

Epoch 37/100
1/1 [==============================] - 0s 99ms/step - loss: 28.5462 - val_loss: 98.5774
Epoch 38/100
1/1 [==============================] - 0s 102ms/step - loss: 28.4893 - val_loss: 94.4085
Epoch 39/100
1/1 [==============================] - 0s 103ms/step - loss: 27.4516 - val_loss: 85.6902
Epoch 40/100
1/1 [==============================] - 0s 100ms/step - loss: 27.0199 - val_loss: 85.4365
Epoch 41/100
1/1 [==============================] - 0s 103ms/step - loss: 26.5639 - val_loss: 93.6987
Epoch 42/100
1/1 [==============================] - 0s 103ms/step - loss: 25.8318 - val_loss: 98.0253
Epoch 43/100
1/1 [==============================] - 0s 99ms/step - loss: 25.6485 - val_loss: 91.9814
Epoch 44/100
1/1 [==============================] - 0s 101ms/step - loss: 24.7930 - val_loss: 87.0222
Epoch 45/100
1/1 [==============================] - 0s 102ms/step - loss: 24.5998 - val_loss: 92.6818
Epoch 46/100
1/1 [==============================] - 0s 101ms/step - loss: 23.7542 - 

Epoch 17/100
1/1 [==============================] - 0s 101ms/step - loss: 27.9147 - val_loss: 43.6553
Epoch 18/100
1/1 [==============================] - 0s 99ms/step - loss: 29.2942 - val_loss: 43.4367
Epoch 19/100
1/1 [==============================] - 0s 103ms/step - loss: 27.8142 - val_loss: 43.0270
Epoch 20/100
1/1 [==============================] - 0s 101ms/step - loss: 25.7578 - val_loss: 43.3676
Epoch 21/100
1/1 [==============================] - 0s 100ms/step - loss: 26.2029 - val_loss: 43.6732
Epoch 22/100
1/1 [==============================] - 0s 102ms/step - loss: 26.3390 - val_loss: 43.6152
Epoch 23/100
1/1 [==============================] - 0s 98ms/step - loss: 24.8640 - val_loss: 43.7034
Epoch 24/100
1/1 [==============================] - 0s 99ms/step - loss: 24.0540 - val_loss: 44.0973
Epoch 25/100
1/1 [==============================] - 0s 103ms/step - loss: 24.6873 - val_loss: 44.0255
Epoch 26/100
1/1 [==============================] - 0s 101ms/step - loss: 23.6056 - v

Epoch 98/100
1/1 [==============================] - 0s 105ms/step - loss: 9.7800 - val_loss: 45.3458
Epoch 99/100
1/1 [==============================] - 0s 103ms/step - loss: 9.4491 - val_loss: 45.2074
Epoch 100/100
1/1 [==============================] - 0s 103ms/step - loss: 9.2821 - val_loss: 45.4028
Train on 1 samples, validate on 1 samples
Epoch 1/100
1/1 [==============================] - 0s 120ms/step - loss: 50.6451 - val_loss: 37.0642
Epoch 2/100
1/1 [==============================] - 0s 114ms/step - loss: 48.9624 - val_loss: 36.3702
Epoch 3/100
1/1 [==============================] - 0s 107ms/step - loss: 45.5651 - val_loss: 36.4491
Epoch 4/100
1/1 [==============================] - 0s 103ms/step - loss: 41.9639 - val_loss: 37.1823
Epoch 5/100
1/1 [==============================] - 0s 106ms/step - loss: 39.5334 - val_loss: 38.0088
Epoch 6/100
1/1 [==============================] - 0s 101ms/step - loss: 38.3697 - val_loss: 38.5242
Epoch 7/100
1/1 [==============================]

Epoch 79/100
1/1 [==============================] - 0s 100ms/step - loss: 10.9987 - val_loss: 41.9872
Epoch 80/100
1/1 [==============================] - 0s 101ms/step - loss: 11.3413 - val_loss: 43.1332
Epoch 81/100
1/1 [==============================] - 0s 101ms/step - loss: 11.2359 - val_loss: 42.3376
Epoch 82/100
1/1 [==============================] - 0s 99ms/step - loss: 10.8836 - val_loss: 42.9119
Epoch 83/100
1/1 [==============================] - 0s 99ms/step - loss: 10.5205 - val_loss: 42.5388
Epoch 84/100
1/1 [==============================] - 0s 100ms/step - loss: 10.3432 - val_loss: 43.0314
Epoch 85/100
1/1 [==============================] - 0s 101ms/step - loss: 10.2853 - val_loss: 42.6409
Epoch 86/100
1/1 [==============================] - 0s 100ms/step - loss: 10.3643 - val_loss: 43.6103
Epoch 87/100
1/1 [==============================] - 0s 103ms/step - loss: 10.6224 - val_loss: 42.6927
Epoch 88/100
1/1 [==============================] - 0s 98ms/step - loss: 10.7159 - v

Epoch 60/100
1/1 [==============================] - 0s 98ms/step - loss: 8.7432 - val_loss: 55.3749
Epoch 61/100
1/1 [==============================] - 0s 100ms/step - loss: 8.5944 - val_loss: 55.5452
Epoch 62/100
1/1 [==============================] - 0s 99ms/step - loss: 8.4787 - val_loss: 55.3623
Epoch 63/100
1/1 [==============================] - 0s 101ms/step - loss: 8.4070 - val_loss: 55.7936
Epoch 64/100
1/1 [==============================] - 0s 105ms/step - loss: 8.4313 - val_loss: 55.0994
Epoch 65/100
1/1 [==============================] - 0s 102ms/step - loss: 8.5825 - val_loss: 55.9781
Epoch 66/100
1/1 [==============================] - 0s 111ms/step - loss: 8.6120 - val_loss: 55.1460
Epoch 67/100
1/1 [==============================] - 0s 103ms/step - loss: 8.1670 - val_loss: 55.6021
Epoch 68/100
1/1 [==============================] - 0s 106ms/step - loss: 7.8685 - val_loss: 55.4325
Epoch 69/100
1/1 [==============================] - 0s 103ms/step - loss: 7.6881 - val_loss: 

In [ ]:
runtime_sec = time.clock() - start
m, s = divmod(runtime_sec, 60)
h, m = divmod(m, 60)
d, h = divmod(h, 24)
print('runtime : %d days %d:%02d:%02d' % (d, h, m, s))

In [ ]:
net.save_weights('./checkpoints/flownetSimple_for_depth.hdf5')